# Process Eu Ai Act and NIST PDS

#### Now we will save each webpages metadata, markdown and html

# Prepare Documents for Ingestion into VectorDB


1. Split the PDF by page and extract its text

In [ ]:
import weaviate
from weaviate.connect import ConnectionParams
from weaviate.classes.init import AdditionalConfig, Timeout
import weaviate.classes.query as wq
import os
from dotenv import load_dotenv
from weaviate.classes.init import Auth

load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize Weaviate connection
client = weaviate.connect_to_local(
        # auth_credentials=weaviate.auth.AuthApiKey("admin-key"),
         # headers={
         #    "X-OpenAI-Api-Key": OPENAI_API_KEY,
         # }
    )

print(client.is_ready())


: 

In [ ]:
COLLECTION_NAME = "gen_ai_knowledge"

: 

In [59]:
client.collections.delete(COLLECTION_NAME)

In [ ]:
import sys
import weaviate
import weaviate.classes.config as wc
import weaviate.classes as wvc
from weaviate.classes.config import Configure
from weaviate.classes.config import Configure, Property, DataType, VectorDistances, VectorFilterStrategy, Multi2VecField

try:
    client.collections.create(
        name=COLLECTION_NAME,
        description="A collection of PDF chunks about Generative AI from various sources like mckinsey, bloomberg, etc. It talks about agentic ai patterns, agentic workflows, agentic agents, mcp, security. It also has information about executive level and mid level strategy and architectures. example use cases. All things related to generative AI",
        properties=[
                wc.Property(name="original_pdf_name", description="Name of the original PDF", data_type=wc.DataType.TEXT),
                wc.Property(name="original_pdf_path", description="Path to the original PDF", data_type=wc.DataType.TEXT),
                wc.Property(name="pdf_name", description="Name of the PDF", data_type=wc.DataType.TEXT),
                wc.Property(name="page_number", description="Page number of the chunk", data_type=wc.DataType.INT),
                wc.Property(name="total_pages", description="Total pages in the original PDF", data_type=wc.DataType.INT),
                wc.Property(name="chunk_png_file", description="Path to the PNG file of the chunk", data_type=wc.DataType.TEXT),
                wc.Property(name="chunk_pdf_file", description="Path to the PDF file of the chunk", data_type=wc.DataType.TEXT),
                wc.Property(name="chunk_text_file", description="Path to the text file of the chunk", data_type=wc.DataType.TEXT),
                wc.Property(name="content", description="Content of the chunk", data_type=wc.DataType.TEXT),
                wc.Property(name="content_markdown", description="Content of the chunk in markdown format", data_type=wc.DataType.TEXT),
                wc.Property(name="text_length", description="Length of the text in the chunk", data_type=wc.DataType.INT),
                wc.Property(name="timestamp", description="Timestamp of the chunk", data_type=wc.DataType.INT),
                wc.Property(name="width", description="Width of the chunk", data_type=wc.DataType.INT),
                wc.Property(name="height", description="Height of the chunk", data_type=wc.DataType.INT),
                wc.Property(name="img_late_interaction_model", description="", data_type=wc.DataType.TEXT),
                wc.Property(name="text_late_interaction_model", description="", data_type=wc.DataType.TEXT),
                wc.Property(name="sentence_transformers_model", description="", data_type=wc.DataType.TEXT),
                # NER/GLiNER properties
                wc.Property(name="entities_model", description="Model used for entity extraction", data_type=wc.DataType.TEXT),
                wc.Property(name="company", description="Company entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="person", description="Person entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="product", description="Product entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="location", description="Location entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="frameworks", description="Framework entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="reader_type", description="Type of reader (e.g., technical, executive)", data_type=wc.DataType.TEXT),
                wc.Property(name="technologies", description="Technology entities extracted from the content", data_type=wc.DataType.TEXT_ARRAY),
                wc.Property(name="type", description="Document type classification", data_type=wc.DataType.TEXT),
                
            ],
        # Configure named vectors for Cohere embeddings
        # vectorizer_config=[
        #     Configure.NamedVectors.none(
        #         name="cohere_vector"
        #     )
        # ],
        # vector_config=[
        #     Configure.Vectors.text2vec_openai(
        #         model="text-embedding-3-small",
        #         dimensions=1536  # Specify embedding dimensions for OpenAI
        #     ),
            
        # ],
        # vector_config=Configure.Vectors.text2vec_transformers(
        #     name="default"
        # ),
        # vector_config=[Configure.NamedVectors.none(
        #         name="colqwen",
        #         vector_index_config=Configure.VectorIndex.hnsw(
        #             multi_vector=Configure.VectorIndex.MultiVector.multi_vector(
        #                 encoding=Configure.VectorIndex.MultiVector.Encoding.muvera(),
        #             ),
        #         )
        #     )]
        vector_config=[
        # User-provided embeddings
            Configure.MultiVectors.self_provided(
                name="img_late_interaction_vector",
            ),
            Configure.MultiVectors.self_provided(
                name="txt_late_interaction_vector",
            ),
            Configure.Vectors.self_provided(
                name="sentence_transformers_embeddings",
            ),
            Configure.Vectors.self_provided(
                name="openai_embeddings",
            ),

        ],
        
        # User-provided embeddings
        
        inverted_index_config=Configure.inverted_index(
            index_null_state=True  # <-- Enables null state indexing
        ),
        # Use OpenAI for vectorization with focus on the text field
        # vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        #     vectorize_collection_name=False,
        #     model="text-embedding-3-small",
        #     dimensions=1536  # Specify embedding dimensions for OpenAI
        # ),
        # Enable generative search with OpenAI
        # generative_config=wc.Configure.Generative.openai(),
        # Configure replication for reliability
        replication_config=Configure.replication(
            factor=1,
        ),
        # Configure sharding for scalability
        sharding_config=Configure.sharding(
            desired_count=1,
        )
    )

    print(f"Collection '{COLLECTION_NAME}' created successfully.")
except Exception as e:
    print(f"An error occurred while creating the collection: {str(e)}")

Collection 'gen_ai_knowledge' created successfully.


In [61]:
def get_all_collections_info(client):
    collection = client.collections.list_all(simple=False)
    for name, config in collection.items():
        collection = client.collections.get(name)
        print(f"--------------{config.name}------------------")
        print(f"Collection Name: {config.name}")
        if not config.multi_tenancy_config.enabled:
            print("Total Documents: ", collection.aggregate.over_all(total_count=True).total_count)
        else:
            tenants = collection.tenants.get()
            print(f"# of Tenants: {len(tenants)}")
        print(f"Description: {config.description}")

In [62]:
get_all_collections_info(client)

--------------AiGovernanceStandardsAndRegulations------------------
Collection Name: AiGovernanceStandardsAndRegulations
Total Documents:  192
Description: A collection of PDF chunks about Generative AI from various sources like mckinsey, bloomberg, etc. It talks about agentic ai patterns, agentic workflows, agentic agents, mcp, security. It also has information about executive level and mid level strategy and architectures. example use cases. All things related to generative AI
--------------EuAiActArticles------------------
Collection Name: EuAiActArticles
Total Documents:  111
Description: A collection of PDF chunks about Generative AI from various sources like mckinsey, bloomberg, etc. It talks about agentic ai patterns, agentic workflows, agentic agents, mcp, security. It also has information about executive level and mid level strategy and architectures. example use cases. All things related to generative AI
--------------Eu_ai_act_nist_pdfs------------------
Collection Name: Eu_

In [63]:
collection = client.collections.get(COLLECTION_NAME)
collection_config = collection.config.get()
for property in collection_config.properties:
    print(f"Field Name: '{property.name}' with type {property.data_type}")
for vector in collection_config.vector_config:
    print(f"Vector Field: {vector}")

# print(json.dumps(collection_config.to_dict(), indent=2))

Field Name: 'original_pdf_name' with type DataType.TEXT
Field Name: 'original_pdf_path' with type DataType.TEXT
Field Name: 'pdf_name' with type DataType.TEXT
Field Name: 'page_number' with type DataType.INT
Field Name: 'total_pages' with type DataType.INT
Field Name: 'chunk_png_file' with type DataType.TEXT
Field Name: 'chunk_pdf_file' with type DataType.TEXT
Field Name: 'chunk_text_file' with type DataType.TEXT
Field Name: 'content' with type DataType.TEXT
Field Name: 'content_markdown' with type DataType.TEXT
Field Name: 'text_length' with type DataType.INT
Field Name: 'timestamp' with type DataType.INT
Field Name: 'width' with type DataType.INT
Field Name: 'height' with type DataType.INT
Field Name: 'img_late_interaction_model' with type DataType.TEXT
Field Name: 'text_late_interaction_model' with type DataType.TEXT
Field Name: 'sentence_transformers_model' with type DataType.TEXT
Vector Field: img_late_interaction_vector
Vector Field: openai_embeddings
Vector Field: sentence_trans

## Ingest the data into Weaviate

In [64]:
collection = client.collections.get(COLLECTION_NAME)

In [65]:
import weaviate
import numpy as np
import json
from pathlib import Path
from tqdm import tqdm
from weaviate.util import generate_uuid5

def ingest_data(client, collection_name: str, source_folder: str):
    root = Path(source_folder)
    collection = client.collections.get(collection_name)
    chunks = []

    # Recursively find all 'metadata.json' files inside the source folder
    # This works whether you pass a specific document folder or the parent 'processed_data' folder
    print(f"Scanning directory: {root} ...")
    metadata_files = list(root.rglob("metadata.json"))

    for meta_file in metadata_files:
        try:
            with open(meta_file, "r", encoding="utf-8") as f:
                chunks.append(json.load(f))
        except Exception as e:
            print(f"Error loading {meta_file}: {e}")

    if not chunks:
        print("No chunks found – check root path / structure.")
        return

    print(f"Ingesting {len(chunks)} chunks...")

    with collection.batch.fixed_size(batch_size=60) as batch:
        for chunk in tqdm(chunks):
            # Pop vectors so they aren't stored as text properties
            img_late_interaction_vector = chunk.pop("img_late_interaction_vector", None)
            txt_late_interaction_vector = chunk.pop("text_late_interaction_vector", None)
            sentence_transformers_embeddings = chunk.pop("sentence_transformers_embeddings", None)
            openai_embeddings = chunk.pop("openai_embeddings", None)
            chunk["width"] = int(chunk["width"])
            chunk["height"] = int(chunk["height"])

            # # If you *require* this embedding, you can skip the object:
            # if sentance_transformers_embeddings is None:
            #     print(
            #         f"Missing embedding for "
            #         f"{chunk.get('original_pdf_name')}-{chunk.get('page_number')}"
            #     )
            #     # Option 1: skip this object entirely
            #     # continue
            #     # Option 2: allow it but without this vector:
            #     # leave sentance_transformers_embeddings as None and drop it below

            # else:
            #     # Your JSON seems to store a list-of-vectors; take the first
            #     sentance_transformers_embeddings = sentance_transformers_embeddings[0]

            # Build vector dict only with non-None entries
            vectors = {}
            if img_late_interaction_vector is not None:
                vectors["img_late_interaction_vector"] = img_late_interaction_vector
            if txt_late_interaction_vector is not None:
                vectors["txt_late_interaction_vector"] = txt_late_interaction_vector
            if sentence_transformers_embeddings is not None:
                # NOTE: key must match the name in vector_config
                vectors["sentence_transformers_embeddings"] = sentence_transformers_embeddings[0]
            if openai_embeddings is not None:
                vectors["openai_embeddings"] = openai_embeddings

            # If you want to allow objects with *no* vectors at all, keep this as-is.
            # If your collection is self_provided() for all vectors, this is allowed.
            if vectors:
                batch.add_object(
                    properties=chunk,
                    uuid=generate_uuid5(
                        f"{chunk['original_pdf_name']}-{chunk['page_number']}"
                    ),
                    vector=vectors,
                )
            else:
                # Object with no vectors (your "null vector" case)
                batch.add_object(
                    properties=chunk,
                    uuid=generate_uuid5(
                        f"{chunk['original_pdf_name']}-{chunk['page_number']}"
                    ),
                    # no `vector=` argument
                )


            if batch.number_errors > 10:
                print("Batch import stopped due to excessive errors.")
                break

    if collection.batch.failed_objects:
        print(f"Failed to import {len(collection.batch.failed_objects)} objects")
        for failed in collection.batch.failed_objects[:3]:
            print(f"Original UUID: {failed.original_uuid}")
            print(f"Error: {failed.message}")
            print(f"Object: {failed.object_.properties}")
    else:
        print("Import successful - no errors")

# Usage Example:


In [66]:
ingest_data(client, COLLECTION_NAME, "./processed_data")

Scanning directory: processed_data ...
Ingesting 8264 chunks...


100%|██████████| 8264/8264 [00:29<00:00, 281.07it/s] 


Import successful - no errors


In [67]:
collection.aggregate.over_all(total_count=True).total_count

8264

In [68]:
get_all_collections_info(client)

--------------AiGovernanceStandardsAndRegulations------------------
Collection Name: AiGovernanceStandardsAndRegulations
Total Documents:  192
Description: A collection of PDF chunks about Generative AI from various sources like mckinsey, bloomberg, etc. It talks about agentic ai patterns, agentic workflows, agentic agents, mcp, security. It also has information about executive level and mid level strategy and architectures. example use cases. All things related to generative AI
--------------EuAiActArticles------------------
Collection Name: EuAiActArticles
Total Documents:  111
Description: A collection of PDF chunks about Generative AI from various sources like mckinsey, bloomberg, etc. It talks about agentic ai patterns, agentic workflows, agentic agents, mcp, security. It also has information about executive level and mid level strategy and architectures. example use cases. All things related to generative AI
--------------Eu_ai_act_nist_pdfs------------------
Collection Name: Eu_

In [69]:
from tracemalloc import Snapshot
import weaviate.classes as wvc
from weaviate.classes.query import Filter, GroupBy
from utils import sentence_transformers_embed

import time
# Start the timer
start_time = time.time()

response = collection.query.fetch_objects(
    limit=1,
    return_metadata=wvc.query.MetadataQuery(distance=True,certainty=True),
    # filters=filters,
    # group_by=group_by,
    include_vector=True,
    # certainty=0.9, # can be between 0 and 1: closer to 1 is more similar
    # distance= 0.2, # can be between 0 and 1: closer to 0 is more similar
)


elapsed_time = time.time() - start_time
print(f"Query execution time: {elapsed_time:.4f} seconds")
# # Process and print the response
for i, o in enumerate(response.objects, start=1):
    # print(f"-----Result {i} ------")
    # for key, value in o.properties.items():
        # print(f"{key}: {value}")
    print(f"Original PDF Name: {o.properties['original_pdf_name']}")
    print(f"Page {o.properties['page_number']}")
    # print(f"Content: {o.properties['content']:20}")
    # print(f"Page Title: {o.properties['page_title']}")
    # print(o.vector)
    for name, vec in o.vector.items():
        print(name, len(vec))
        print(vec)
    # print(vec)  # beware: huge
    # print(f"Distance: {o.metadata.distance}")
    # print(f"Certainty: {o.metadata.certainty}")
    # dimensionality = len(o.vector["multi_vector"])
    # print("dimensionality length is", dimensionality)
    print("--------------------------------")

Query execution time: 0.0220 seconds
Original PDF Name: Stanford_ai_index_report_2025
Page 231
sentence_transformers_embeddings 4096
[-0.002033175202086568, -0.01687243953347206, 0.008522136136889458, -0.0023164195008575916, 0.003668535500764847, 0.015501974150538445, 0.031236059963703156, -0.0016814591363072395, 0.004915338475257158, 0.007957211695611477, -0.022302353754639626, -0.0024158977903425694, -0.018282055854797363, 0.0031626212876290083, 0.006843206938356161, -0.005352779291570187, -0.008365171030163765, 0.003660745220258832, -0.026560019701719284, -0.001335495850071311, -0.01583727076649666, 0.017595646902918816, 0.0074196383357048035, 0.03035311959683895, -0.009076991118490696, 0.016350753605365753, -0.020659198984503746, -0.0034246654249727726, 0.025829937309026718, 0.0037797819823026657, -0.0105366175994277, 0.029297761619091034, 0.00764654902741313, 0.015021991916000843, 0.0036957599222660065, -0.0043442752212285995, 0.04145172983407974, 0.011025415733456612, 0.023783273

In [70]:
from tracemalloc import Snapshot
import weaviate.classes as wvc
from weaviate.classes.query import Filter, GroupBy
from utils import sentence_transformers_embed, openai_embed


# collection = client.collections.get("wtf")
QUERY = "agents"
import time
# Start the timer
start_time = time.time()

# query_vector = sentance_transformers_embed(QUERY)[0]
# print(query_vector)
# print(len(query_vector))

response = collection.query.bm25(
    query=QUERY,
    
    limit=10,
    return_metadata=wvc.query.MetadataQuery(score=True,certainty=True),
    # filters=filters,
    # group_by=group_by,
    include_vector=True,
    # certainty=0.9, # can be between 0 and 1: closer to 1 is more similar
    # distance= 0.2, # can be between 0 and 1: closer to 0 is more similar

)


elapsed_time = time.time() - start_time
print(f"Query: {QUERY}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
print(f"Length of Returned Objects: {len(response.objects)}")
# # Process and print the response
for i, o in enumerate(response.objects, start=1):
    # print(f"-----Result {i} ------")
    for key, value in o.properties.items():
        print(f"{key}: {value}")
    print(f"Original PDF Name: {o.properties['original_pdf_name']}")
    print(f"Page {o.properties['page_number']}")
    # print(f"Content: {o.properties['content']:20}")
    # print(f"Page Title: {o.properties['page_title']}")
    # print(o.vector)
    for vector in o.vector:
        print(f"Vector: {vector}")
        print(type(vector))
    print(f"Distance: {o.metadata.distance}")
    # print(f"Certainty: {o.metadata.certainty}")
    # dimensionality = len(o.vector["multi_vector"])
    # print("dimensionality length is", dimensionality)
    print("--------------------------------")

Query: agents
Query execution time: 0.1151 seconds
Length of Returned Objects: 10
original_pdf_name: N8N_AI_Agent_Explained_Guide_1753301982
chunk_pdf_file: N8N_AI_Agent_Explained_Guide_1753301982/page_2/page_2.pdf
page_number: 2
chunk_text_file: N8N_AI_Agent_Explained_Guide_1753301982/page_2/text.txt
text_length: 834
img_late_interaction_model: None
pdf_name: N8N_AI_Agent_Explained_Guide_1753301982_2
sentence_transformers_model: nvidia/llama-embed-nemotron-8b
total_pages: 3
height: 1012
chunk_png_file: N8N_AI_Agent_Explained_Guide_1753301982/page_2/page_2.png
content: Introduction
What are AI agents?
How do AI agents work?
Input processing, Decision-making, Action execution
Learning and adaptation
What are the types of agents in AI?
Simple reflex agents, Model-based reflex agents
Goal-based agents
Utility-based agents, Learning agents
What are the benefits of AI agents?
Faster information analysis and decision-making
Increased team productivity
Enhanced customer experience
Accelerated

In [76]:
from tracemalloc import Snapshot
import weaviate.classes as wvc
from weaviate.classes.query import Filter, GroupBy
from utils import sentence_transformers_embed, openai_embed


# collection = client.collections.get("wtf")
QUERY = "agent use cases in manufactoring thinking strategically"
import time
# Start the timer
start_time = time.time()

query_vector = sentance_transformers_embed(QUERY)[0]
# print(query_vector)
# print(len(query_vector))

response = collection.query.near_vector(
    near_vector=query_vector,
    target_vector=["sentence_transformers_embeddings"],
    limit=10,
    return_metadata=wvc.query.MetadataQuery(distance=True,certainty=True),
    # filters=filters,
    # group_by=group_by,
    include_vector=True,
    # certainty=0.9, # can be between 0 and 1: closer to 1 is more similar
    # distance= 0.2, # can be between 0 and 1: closer to 0 is more similar

)


elapsed_time = time.time() - start_time
print(f"Query: {QUERY}")
print(f"Query execution time: {elapsed_time:.4f} seconds")
print(f"Length of Returned Objects: {len(response.objects)}")
# # Process and print the response
for i, o in enumerate(response.objects, start=1):
    # print(f"-----Result {i} ------")
    # for key, value in o.properties.items():
        # print(f"{key}: {value}")
    print(f"Original PDF Name: {o.properties['original_pdf_name']}")
    print(f"Page {o.properties['page_number']}")
    print(o.properties['content'][:1000])
    # print(f"Content: {o.properties['content']:20}")
    # print(f"Page Title: {o.properties['page_title']}")
    # print(o.vector)
    print(f"Distance: {o.metadata.distance}")
    # print(f"Certainty: {o.metadata.certainty}")
    # dimensionality = len(o.vector["multi_vector"])
    # print("dimensionality length is", dimensionality)
    print("--------------------------------")

Query: agent use cases in manufactoring thinking strategically
Query execution time: 1.6512 seconds
Length of Returned Objects: 10
Original PDF Name: Anthropic_2026_State_of_AI_Agents_Report
Page 23
AI agents in 
production

Distance: 0.42092013359069824
--------------------------------
Original PDF Name: Anthropic_New_2026_State_of_AI_Agents_1765636542
Page 23
AI agents in 
production

Distance: 0.42092013359069824
--------------------------------
Original PDF Name: Deloitte-in-eaid-agentic-ai-pov-chapter-1
Page 2
Unlocking the right agentic AI use cases
2

Distance: 0.4258580207824707
--------------------------------
Original PDF Name: Deloitte_Unlocking_the_right_agentic_AI_use_cases_1761432502
Page 2
Unlocking the right agentic AI use cases
2

Distance: 0.4258580207824707
--------------------------------
Original PDF Name: Deloitte-in-eaid-agentic-ai-pov-chapter-1
Page 15
Unlocking the right agentic AI use cases
15

Distance: 0.43404197692871094
--------------------------------
Ori